In [1]:
import requests
from bs4 import BeautifulSoup
import json
import time # Dùng để sleep chương trình
import pandas as pd # Dùng để đọc và hiển thị file csv (Pandas sẽ được học chi tiết ở buổi tới)
import datetime as dt # Dùng để xử lý dữ liệu thời gian
import re

In [60]:
Total_pages=10
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}

with requests.Session() as s:
    page =1
    url = 'https://www.zillow.com/manchester-nh/sold/?searchQueryState=%7B"pagination"%3A%7B%7D%2C"mapBounds"%3A%7B"west"%3A-71.53618812561035%2C"east"%3A-71.34221076965332%2C"south"%3A42.96100798811988%2C"north"%3A43.06192743848626%7D%2C"regionSelection"%3A%5B%7B"regionId"%3A5772%2C"regionType"%3A6%7D%5D%2C"isMapVisible"%3Atrue%2C"filterState"%3A%7B"sort"%3A%7B"value"%3A"globalrelevanceex"%7D%2C"fsba"%3A%7B"value"%3Afalse%7D%2C"fsbo"%3A%7B"value"%3Afalse%7D%2C"nc"%3A%7B"value"%3Afalse%7D%2C"fore"%3A%7B"value"%3Afalse%7D%2C"cmsn"%3A%7B"value"%3Afalse%7D%2C"auc"%3A%7B"value"%3Afalse%7D%2C"pmf"%3A%7B"value"%3Afalse%7D%2C"pf"%3A%7B"value"%3Afalse%7D%2C"rs"%3A%7B"value"%3Atrue%7D%2C"ah"%3A%7B"value"%3Atrue%7D%7D%2C"isListVisible"%3Atrue%2C"mapZoom"%3A13%7D'
    while(page < Total_pages):
        time.sleep(1)
        r = s.get(url, headers=req_headers)
        soup = BeautifulSoup(r.content, 'html.parser')
        detailLink = soup.find_all('a',{'class':'list-card-link list-card-link-top-margin'})
        prices = soup.find_all('div',{'class':'list-card-price'})
        page+=1
        url ='https://www.zillow.com/manchester-nh/sold/' +str(page)+'_p/'
        for link in detailLink:
            print(link)
            address = link.text
            detail = s.get(link['href'],headers=req_headers)
            soup1 = BeautifulSoup(detail.content, 'html.parser')
            bbl = soup1.find_all('span',{'class':'ds-bed-bath-living-area'},limit =3) # bed-bad-livingarea
            bed_bad_livingarea =''
            for i in bbl:
                bed_bad_livingarea+= i.text +'\t'
            print(bed_bad_livingarea)
            heating = soup1.find("span",string=re.compile("Heating:"))
            if heating is not None:
                heating = heating.find_next_sibling('span').text
                print(heating)
            basement = soup1.find("span",string=re.compile("Basement"))
            if basement is not None:
                if ':' in basement.find_previous('span'):
                    basement = basement.find_next('span').text.split(': ')[1]
                    print(basement)
            yearbuild = soup1.find("span",string=re.compile("Condition"))
            if yearbuild is not None:
                if ':' in yearbuild.find_previous('span'):
                    yearbuild = yearbuild.find_next('span').text.split(': ')[1]
                    print(yearbuild)
            fireplace = soup1.find("h5",string=re.compile("Property details"))
            if fireplace is not None:
                if ':' in fireplace.find_previous('span'):
                    fireplace = fireplace.find_previous('span').text.split(': ')[1]
                    print(fireplace)
            garage = soup1.find("span",string=re.compile("Property"))
            if garage is not None:
                if ':' in garage.find_previous('span'):
                    garage = garage.find_previous('span').text.split(': ')[1]
                    print(garage)
            #fw.write(address +'\t'+ bed_bad_livingarea +'\t'+ provider +'\t'+ rating +'\t'+ num_ratings + '\n')

<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/55-River-Rd-APT-7A-Manchester-NH-03104/95349869_zpid/" tabindex="0"><address class="list-card-addr">55 River Rd APT 7A, Manchester, NH 03104</address></a>
2 bd	2 ba	761 sqft	
Baseboard, Electric
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/44-Maryland-Ave-Manchester-NH-03104/74285748_zpid/" tabindex="0"><address class="list-card-addr">44 Maryland Ave, Manchester, NH 03104</address></a>
2 bd	1 ba	1,870 sqft	
Electric, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/10-Estate-Dr-Manchester-NH-03104/74321535_zpid/" tabindex="0"><address class="list-card-addr">10 Estate Dr, Manchester, NH 03104</address></a>
3 bd	2 ba	1,597 sqft	
Electric, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/415-Calef-Rd-Manchester-NH-03103/74313545_zpid/" tabindex="0"><addres

6 bd	4 ba	3,178 sqft	
Stove, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/470-Silver-St-APT-302-Manchester-NH-03103/119151027_zpid/" tabindex="0"><address class="list-card-addr">470 Silver St APT 302, Manchester, NH 03103</address></a>
2 bd	1 ba	768 sqft	
Forced air, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/195-Comeau-St-3-Manchester-NH-03102/2077248128_zpid/" tabindex="0"><address class="list-card-addr">195 Comeau St #3, Manchester, NH 03102</address></a>
2 bd	1 ba	1,100 sqft	
Electric, Wood / Pellet
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/397-Cedar-St-Manchester-NH-03103/74310273_zpid/" tabindex="0"><address class="list-card-addr">397 Cedar St, Manchester, NH 03103</address></a>
6 bd	2 ba	2,544 sqft	
Baseboard, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/597-Beech-St-Manche

3 bd	1 ba	1,358 sqft	
Baseboard, Electric, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/21-Sewall-St-Manchester-NH-03103/74316609_zpid/" tabindex="0"><address class="list-card-addr">21 Sewall St, Manchester, NH 03103</address></a>
3 bd	1 ba	1,116 sqft	
Baseboard, Forced air, Electric, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/28-Blueberry-Dr-Manchester-NH-03102/95345522_zpid/" tabindex="0"><address class="list-card-addr">28 Blueberry Dr, Manchester, NH 03102</address></a>
3 bd	3 ba	2,284 sqft	
Forced air, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/292-Walnut-St-Manchester-NH-03104/74288688_zpid/" tabindex="0"><address class="list-card-addr">292 Walnut St, Manchester, NH 03104</address></a>
6 bd	-- ba	3,021 sqft	
Electric, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/1472-Belmo

3 bd	2 ba	2,140 sqft	
Electric
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/175-3rd-St-Manchester-NH-03102/74280579_zpid/" tabindex="0"><address class="list-card-addr">175 3rd St, Manchester, NH 03102</address></a>
2 bd	1 ba	1,104 sqft	
Forced air, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/75-Log-St-APT-1H-Manchester-NH-03102/95346163_zpid/" tabindex="0"><address class="list-card-addr">75 Log St APT 1H, Manchester, NH 03102</address></a>
2 bd	1 ba	728 sqft	
Baseboard, Electric, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/153-Norton-Ave-Manchester-NH-03109/74289210_zpid/" tabindex="0"><address class="list-card-addr">153 Norton Ave, Manchester, NH 03109</address></a>
4 bd	1 ba	1,104 sqft	
Baseboard, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/108-Riverwalk-Way-Manchester-NH-03101/11

3 bd	3 ba	2,408 sqft	
Forced air, Electric, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/70-Newgate-Cir-Manchester-NH-03102/74317314_zpid/" tabindex="0"><address class="list-card-addr">70 Newgate Cir, Manchester, NH 03102</address></a>
3 bd	2 ba	2,514 sqft	
Electric, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/159-Brunelle-Ave-Manchester-NH-03103/74297124_zpid/" tabindex="0"><address class="list-card-addr">159 Brunelle Ave, Manchester, NH 03103</address></a>
2 bd	2 ba	1,344 sqft	
Forced air, Electric, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/147-Leewood-St-Manchester-NH-03103/74299089_zpid/" tabindex="0"><address class="list-card-addr">147 Leewood St, Manchester, NH 03103</address></a>
4 bd	3 ba	1,928 sqft	
Baseboard, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/276-Tarrytown-

...

2 bd	1 ba	1,020 sqft	
Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/205-Cameron-St-Manchester-NH-03103/74319739_zpid/" tabindex="0"><address class="list-card-addr">205 Cameron St, Manchester, NH 03103</address></a>
3 bd	2 ba	1,322 sqft	
Baseboard, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/41-Hevey-St-Manchester-NH-03102/119149329_zpid/" tabindex="0"><address class="list-card-addr">41 Hevey St, Manchester, NH 03102</address></a>
-- bd	-- ba	3,617 sqft	
Baseboard, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/213-Auburn-St-Manchester-NH-03103/189826217_zpid/" tabindex="0"><address class="list-card-addr">213 Auburn St, Manchester, NH 03103</address></a>
8 bd	4 ba	4,570 sqft	
Forced air
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/121-Michigan-Ave-Manchester-NH-03104/74287142_zpid/" tabin

3 bd	2 ba	1,704 sqft	
Baseboard, Electric, Gas, Solar
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/50-Riverdale-Ave-Manchester-NH-03103/74319287_zpid/" tabindex="0"><address class="list-card-addr">50 Riverdale Ave, Manchester, NH 03103</address></a>
3 bd	1 ba	1,904 sqft	
Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/1602-Front-St-UNIT-6-Manchester-NH-03102/114777092_zpid/" tabindex="0"><address class="list-card-addr">1602 Front St UNIT 6, Manchester, NH 03102</address></a>
2 bd	2 ba	1,972 sqft	
Forced air, Electric, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/444-Varney-St-Manchester-NH-03102/74311735_zpid/" tabindex="0"><address class="list-card-addr">444 Varney St, Manchester, NH 03102</address></a>
3 bd	2 ba	2,114 sqft	
Baseboard, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/281-Dub

5 bd	2 ba	1,862 sqft	
Forced air, Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/430-Vinton-St-Manchester-NH-03103/74296746_zpid/" tabindex="0"><address class="list-card-addr">430 Vinton St, Manchester, NH 03103</address></a>
2 bd	1 ba	1,260 sqft	
Forced air, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/164-Morse-Rd-Manchester-NH-03104/74322438_zpid/" tabindex="0"><address class="list-card-addr">164 Morse Rd, Manchester, NH 03104</address></a>
2 bd	1 ba	1,090 sqft	
Forced air, Propane / Butane
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/225-Brickett-Rd-Manchester-NH-03109/74325853_zpid/" tabindex="0"><address class="list-card-addr">225 Brickett Rd, Manchester, NH 03109</address></a>
3 bd	2 ba	1,410 sqft	
Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/190-Anthony-St-Manchester-NH-03103/743

1 bd	1 ba	1,011 sqft	
Electric, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/244-Hall-St-Manchester-NH-03103/74295744_zpid/" tabindex="0"><address class="list-card-addr">244 Hall St, Manchester, NH 03103</address></a>
3 bd	2 ba	1,828 sqft	
Electric, Gas, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/60-Charles-Chase-Way-Manchester-NH-03104/95349977_zpid/" tabindex="0"><address class="list-card-addr">60 Charles Chase Way, Manchester, NH 03104</address></a>
3 bd	3 ba	2,165 sqft	
Forced air, Electric, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/34-Depot-Rd-Manchester-NH-03103/74318566_zpid/" tabindex="0"><address class="list-card-addr">34 Depot Rd, Manchester, NH 03103</address></a>
2 bd	1 ba	1,554 sqft	
Forced air, Electric, Oil
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/881-Mammoth-Rd

3 bd	2 ba	1,638 sqft	
Gas
<a class="list-card-link list-card-link-top-margin" href="https://www.zillow.com/homedetails/184-Woodview-Way-Manchester-NH-03102/119150842_zpid/" tabindex="0"><address class="list-card-addr">184 Woodview Way, Manchester, NH 03102</address></a>
2 bd	2.5 ba	1,672 sqft	
Forced air
